# Lollipop plot analysis

## Requires having run vcf2maf on the output of vk clean with a command like this:
- perl vcf2maf.pl --input-vcf variants.vcf --output-maf variants.maf --ref-fasta /path/to/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --cache-version 93 --species homo_sapiens --ncbi-build GRCh37 --retain-info DP,NS

In [1]:
# try:
#     import varseek as vk
# except ImportError:
#     print("varseek not found, installing...")
#     !pip install -U -q varseek
# try:
#     import RLSRWP_2025
# except ImportError:
#     print("RLSRWP_2025 not found, installing...")
#     !pip install -q git+https://github.com/pachterlab/RLSRWP_2025.git

In [1]:
import os
from rpy2.robjects import pandas2ri, r, globalenv
import pandas as pd
import sys
from varseek.utils import download_box_url  # for functions used in both varseek and here
# from RLSRWP_2025 import ...  # for functions only used here

RLSRWP_2025_dir = os.path.dirname(os.path.abspath(""))  # if this notebook resides in RLSRWP_2025/notebooks/0_data_download.ipynb, then this retrieves RLSRWP_2025

### File path definitions and imports

In [ ]:
sorting_method = "number_of_distinct_variants"  # "total_counts" or "number_of_distinct_variants" or "total_samples"
number_of_genes = 5  # int or True (for all)
show_mutations = 3  # int or True (for all) or None (for none)
displayed_counts = "DP"  # "NS" (number of samples, default maftools) or "DP" (counts across all samples)

vk_count_out_dir = os.path.join(RLSRWP_2025_dir, "data", "vk_count_out_fig1")
# maf_path = os.path.join(vk_count_out_dir, "variants.maf")  #!!! uncomment
maf_path = "/Users/joeyrich/Desktop/local/varseek/variants4.maf"  #!!! erase

out_dir = os.path.join(vk_count_out_dir, "analysis", "lollipop_plots4")

### Download the MAF file

In [4]:
if not os.path.exists(maf_path):
    maf_url = ""  #!!!
    download_box_url(maf_url, output_file_name=maf_path)

### Load in rpy2

In [26]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [27]:
# Activate the automatic conversion between pandas and R objects
pandas2ri.activate()

### Load in maftools

In [28]:
%%R
library(maftools)

### Customize the lollipopPlot function from maftools to ensure counts display correctly
Maftools normally displays counts based on the number of samples containing a mutation, where the occurrence of a mutation gets a separate row for each sample in which it occurs, and maftools adds up the number of rows. But if displayed_counts == "DP", then I must use the DP column from the VCF for counts.

In [ ]:
%%R -i displayed_counts
lollipopPlot_custom <- function (maf, data = NULL, gene = NULL, AACol = NULL, labelPos = NULL, 
    labPosSize = 0.9, showMutationRate = TRUE, showDomainLabel = TRUE, 
    cBioPortal = FALSE, refSeqID = NULL, proteinID = NULL, roundedRect = TRUE, 
    repel = FALSE, collapsePosLabel = TRUE, showLegend = TRUE, 
    legendTxtSize = 0.8, labPosAngle = 0, domainLabelSize = 0.8, 
    axisTextSize = c(1, 1), printCount = FALSE, colors = NULL, 
    domainAlpha = 1, domainBorderCol = "black", bgBorderCol = "black", 
    labelOnlyUniqueDoamins = TRUE, defaultYaxis = FALSE, titleSize = c(1.2, 
        1), pointSize = 1.5, displayed_counts="NS") 
{
    if (is.null(gene)) {
        stop("Please provide a gene name.")
    }
    prot = maftools:::.getdomains(geneID = gene, refSeqID = refSeqID, proteinID = proteinID)
    len = as.numeric(max(prot$aa.length, na.rm = TRUE))
    prot = prot[, `:=`(domain_lenght, End - Start)][order(domain_lenght, 
        decreasing = TRUE)][, `:=`(domain_lenght, NULL)]
    cbioSubTitle = gene
    if (is.null(data)) {
        mut = subsetMaf(maf = maf, includeSyn = FALSE, genes = gene,
            query = "Variant_Type != 'CNV'", mafObj = FALSE)
        if (is.null(AACol)) {
            pchange = c("HGVSp_Short", "Protein_Change", "AAChange")
            if (length(pchange[pchange %in% colnames(mut)]) > 
                0) {
                pchange = suppressWarnings(pchange[pchange %in% 
                    colnames(mut)][1])
                message(paste0("Assuming protein change information are stored under column ", 
                    pchange, ". Use argument AACol to override if necessary."))
                colnames(mut)[which(colnames(mut) == pchange)] = "AAChange_"
            }
            else {
                message("Available fields:")
                print(colnames(mut))
                stop("AAChange field not found in MAF. Use argument AACol to manually specifiy field name containing protein changes.")
            }
        }
        else {
            if (length(which(colnames(mut) == AACol)) == 0) {
                message("Available fields:")
                print(colnames(mut))
                stop(paste0("Column ", AACol, " not found."))
            }
            else {
                colnames(mut)[which(colnames(mut) == AACol)] = "AAChange_"
            }
        }
        prot.dat = mut[Hugo_Symbol %in% gene, .(Variant_Type, 
            Variant_Classification, AAChange_)]
        if (nrow(prot.dat) == 0) {
            stop(paste(gene, "does not seem to have any mutations!", 
                sep = " "))
        }
        sampleSize = as.numeric(maf@summary[ID %in% "Samples", 
            summary])
        mutRate = round(getGeneSummary(x = maf)[Hugo_Symbol %in% 
            gene, MutatedSamples]/sampleSize * 100, digits = 2)
        if (showMutationRate) {
            cbioSubTitle = substitute(paste(italic(cbioSubTitle), 
                " : [Somatic Mutation Rate: ", mutRate, "%]"))
        }
        prot.spl = strsplit(x = as.character(prot.dat$AAChange_), 
            split = ".", fixed = TRUE)
        prot.conv = sapply(sapply(prot.spl, function(x) x[length(x)]), 
            "[", 1)
        prot.dat[, `:=`(conv, prot.conv)]
        pos = gsub(pattern = "Ter.*", replacement = "", x = prot.dat$conv)
        pos = gsub(pattern = "[[:alpha:]]", replacement = "", 
            x = pos)
        pos = gsub(pattern = "\\*$", replacement = "", x = pos)
        pos = gsub(pattern = "^\\*", replacement = "", x = pos)
        pos = gsub(pattern = "\\*.*", replacement = "", x = pos)
        pos = as.numeric(sapply(X = strsplit(x = pos, split = "_", 
            fixed = TRUE), FUN = function(x) x[1]))
        prot.dat[, `:=`(pos, abs(pos))]
        if (nrow(prot.dat[is.na(pos)]) > 0) {
            message(paste("Removed", nrow(prot.dat[is.na(prot.dat$pos), 
                ]), "mutations for which AA position was not available", 
                sep = " "))
            prot.dat = prot.dat[!is.na(pos)]
        }
        prot.snp.sumamry = prot.dat[, .N, .(Variant_Classification, 
            conv, pos)]
        colnames(prot.snp.sumamry)[ncol(prot.snp.sumamry)] = "count"
        #!!! JMR
        if (displayed_counts == "DP") {
            mut$AAChange_trimmed <- substr(mut$AAChange_, 3, nchar(mut$AAChange_))  # trim off the 'p.' in front of each mutation in AAChange
            prot.snp.sumamry <- merge(
                prot.snp.sumamry, 
                mut[, .(AAChange_trimmed, DP)],  # Select only needed columns
                by.x = "conv", 
                by.y = "AAChange_trimmed", 
                all.x = TRUE  # Keep all rows from prot.snp.sumamry
            )  # merge by mutation ID
            prot.snp.sumamry$count <- ifelse(is.na(prot.snp.sumamry$DP), 0, prot.snp.sumamry$DP)  # assign DP to count column
            prot.snp.sumamry[, DP := NULL]  # drop DP column
            prot.snp.sumamry <- unique(prot.snp.sumamry)  # drop duplicate rows - important if multiple mutations have the same position
        } else if (displayed_counts == "NS") {
            mut$AAChange_trimmed <- substr(mut$AAChange_, 3, nchar(mut$AAChange_))  # trim off the 'p.' in front of each mutation in AAChange
            prot.snp.sumamry <- merge(
                prot.snp.sumamry, 
                mut[, .(AAChange_trimmed, NS)],  # Select only needed columns
                by.x = "conv", 
                by.y = "AAChange_trimmed", 
                all.x = TRUE  # Keep all rows from prot.snp.sumamry
            )  # merge by mutation ID
            prot.snp.sumamry$count <- ifelse(is.na(prot.snp.sumamry$NS), 0, prot.snp.sumamry$NS)  # assign NS to count column
            prot.snp.sumamry[, NS := NULL]  # drop NS column
            prot.snp.sumamry <- unique(prot.snp.sumamry)  # drop duplicate rows - important if multiple mutations have the same position
        }
        #!!! JMR
    }
    else {
        prot.snp.sumamry = data.table::as.data.table(x = data)
        colnames(prot.snp.sumamry)[1:2] = c("pos", "count")
        if (!"Variant_Classification" %in% colnames(prot.snp.sumamry)) {
            prot.snp.sumamry$Variant_Classification = "variant"
        }
        if (!"conv" %in% colnames(prot.snp.sumamry)) {
            prot.snp.sumamry$conv = prot.snp.sumamry$pos
        }
    }
    if (cBioPortal) {
        vc = c("Nonstop_Mutation", "Frame_Shift_Del", "Missense_Mutation", 
            "Nonsense_Mutation", "Splice_Site", "Frame_Shift_Ins", 
            "In_Frame_Del", "In_Frame_Ins")
        vc.cbio = c("Truncating", "Truncating", "Missense", 
            "Truncating", "Truncating", "Truncating", "In-frame", 
            "In-frame")
        names(vc.cbio) = vc
        col = grDevices::adjustcolor(col = c("black", "#33A02C", 
            "brown"), alpha.f = 0.7)
        col = c(Truncating = col[1], Missense = col[2], `In-frame` = col[3])
    }
    else {
        if (is.null(colors)) {
            col = maftools:::get_vcColors(alpha = 0.7, named = TRUE)
            col["variant"] = "#535C68B3"
        }
        else {
            col = colors
        }
    }
    maxCount = max(prot.snp.sumamry$count, na.rm = TRUE)
    prot.snp.sumamry = prot.snp.sumamry[order(pos), ]
    if (cBioPortal) {
        prot.snp.sumamry$Variant_Classification = vc.cbio[as.character(prot.snp.sumamry$Variant_Classification)]
    }
    if (maxCount <= 5) {
        prot.snp.sumamry$count2 = 1 + prot.snp.sumamry$count
        lim.pos = 2:6
        lim.lab = 1:5
    }
    else {
        prot.snp.sumamry$count2 = 1 + (prot.snp.sumamry$count * 
            (5/max(prot.snp.sumamry$count)))
        lim.pos = prot.snp.sumamry[!duplicated(count2), count2]
        lim.lab = prot.snp.sumamry[!duplicated(count2), count]
    }
    if (length(lim.pos) > 6) {
        lim.dat = data.table::data.table(pos = lim.pos, lab = lim.lab)
        lim.dat[, `:=`(posRounded, round(pos))]
        lim.dat = lim.dat[!duplicated(posRounded)]
        lim.pos = lim.dat[, pos]
        lim.lab = lim.dat[, lab]
    }
    if (!defaultYaxis) {
        lim.pos = c(min(lim.pos), max(lim.pos))
        lim.lab = c(min(lim.lab), max(lim.lab))
    }
    clusterSize = 10
    if (repel) {
        prot.snp.sumamry = repelPoints(dat = prot.snp.sumamry, 
            protLen = len, clustSize = clusterSize)
    }
    else {
        prot.snp.sumamry$pos2 = prot.snp.sumamry$pos
    }
    xlimPos = pretty(0:max(prot$aa.length, na.rm = TRUE))
    xlimPos[length(xlimPos)] = max(prot$aa.length)
    if (!is.null(labelPos)) {
        prot.snp.sumamry = data.table::data.table(prot.snp.sumamry)
        if (length(labelPos) == 1) {
            if (labelPos != "all") {
                prot.snp.sumamry$labThis = ifelse(test = prot.snp.sumamry$pos %in% 
                    labelPos, yes = "yes", no = "no")
                labDat = prot.snp.sumamry[labThis %in% "yes"]
            }
            else {
                labDat = prot.snp.sumamry
            }
        }
        else {
            prot.snp.sumamry$labThis = ifelse(test = prot.snp.sumamry$pos %in% 
                labelPos, yes = "yes", no = "no")
            labDat = prot.snp.sumamry[labThis %in% "yes"]
        }
        if (nrow(labDat) == 0) {
            message(paste0("Position ", labelPos, " doesn't seem to be mutated. Here are the mutated foci."))
            print(prot.snp.sumamry[, .(pos, conv, count, Variant_Classification)][order(pos)])
            stop()
        }
        if (collapsePosLabel) {
            uniquePos = unique(labDat[, pos2])
            labDatCollapsed = data.table::data.table()
            for (i in 1:length(uniquePos)) {
                uniqueDat = labDat[pos2 %in% uniquePos[i]]
                if (nrow(uniqueDat) > 1) {
                    maxDat = max(uniqueDat[, count2])
                    maxPos = unique(uniqueDat[, pos2])
                    toLabel = uniqueDat[, conv]
                    toLabel = paste(toLabel[1], paste(gsub(pattern = "^[A-z]*[[:digit:]]*", 
                        replacement = "", x = toLabel[2:length(toLabel)]), 
                        collapse = "/"), sep = "/")
                    labDatCollapsed = rbind(labDatCollapsed, data.table::data.table(pos2 = maxPos, 
                        count2 = maxDat, conv = toLabel))
                }
                else {
                    labDatCollapsed = rbind(labDatCollapsed, data.table::data.table(pos2 = uniqueDat[, 
                        pos2], count2 = uniqueDat[, count2], conv = uniqueDat[, 
                        conv]))
                }
            }
            labDat = labDatCollapsed
        }
    }
    domains = unique(prot[, Label])
    domain_cols = maftools:::get_domain_cols()
    if (length(domains) > length(domain_cols)) {
        domain_cols = sample(colours(), size = length(domains), 
            replace = FALSE)
    }
    domain_cols = domain_cols[1:length(domains)]
    domain_cols = grDevices::adjustcolor(col = domain_cols, 
        alpha.f = domainAlpha)
    names(domain_cols) = domains
    col = col[unique(as.character(prot.snp.sumamry[, Variant_Classification]))]
    if (showLegend) {
        lo = matrix(data = c(1, 1, 2, 2), nrow = 2, byrow = TRUE)
        graphics::layout(mat = lo, heights = c(4, 1.25))
        par(mar = c(1, 2.5, 2, 1))
    }
    else {
        par(mar = c(2.5, 2.5, 2, 1))
    }
    plot(0, 0, pch = NA, ylim = c(0, 6.5), xlim = c(0, len), 
        axes = FALSE, xlab = NA, ylab = NA)
    rect(xleft = 0, ybottom = 0.2, xright = len, ytop = 0.8, 
        col = "#95a5a6", border = bgBorderCol)
    axis(side = 1, at = xlimPos, labels = xlimPos, lwd = 1.2, 
        font = 1, cex.axis = axisTextSize[1], line = -0.4)
    axis(side = 2, at = lim.pos, labels = lim.lab, lwd = 1.2, 
        font = 1, las = 2, cex.axis = axisTextSize[2])
    segments(x0 = prot.snp.sumamry[, pos2], y0 = 0.8, x1 = prot.snp.sumamry[, 
        pos2], y1 = prot.snp.sumamry[, count2 - 0.03], lwd = 1.2, 
        col = "gray70")
    point_cols = col[as.character(prot.snp.sumamry$Variant_Classification)]
    points(x = prot.snp.sumamry[, pos2], y = prot.snp.sumamry[, 
        count2], col = point_cols, pch = 16, cex = pointSize)
    prot[, `:=`(domainCol, domain_cols[prot[, Label]])]
    if (roundedRect) {
        if (requireNamespace("berryFunctions", quietly = TRUE)) {
            for (i in 1:nrow(prot)) {
                berryFunctions::roundedRect(xleft = prot[i, 
                    Start], ybottom = 0.1, xright = prot[i, End], 
                    ytop = 0.9, col = prot[i, domainCol], border = domainBorderCol, 
                    rounding = 0.08)
            }
        }
        else {
            rect(xleft = prot[, Start], ybottom = 0.1, xright = prot[, 
                End], ytop = 0.9, col = prot[, domainCol], border = domainBorderCol)
        }
    }
    else {
        rect(xleft = prot[, Start], ybottom = 0.1, xright = prot[, 
            End], ytop = 0.9, col = prot[, domainCol], border = domainBorderCol)
    }
    title(main = cbioSubTitle, adj = 0, font.main = 2, cex.main = titleSize[1], 
        line = 0.8)
    title(main = unique(prot[, refseq.ID]), adj = 0, font.main = 1, 
        line = -0.5, cex.main = titleSize[2])
    if (showDomainLabel) {
        if (labelOnlyUniqueDoamins) {
            prot = prot[!duplicated(Label)]
        }
        prot$pos = rowMeans(x = prot[, .(Start, End)], na.rm = FALSE)
        text(y = 0.5, x = prot$pos, labels = prot$Label, font = 3, 
            cex = domainLabelSize)
    }
    if (!is.null(labelPos)) {
        text(x = labDat[, pos2], y = labDat[, count2 + 0.45], 
            labels = labDat[, conv], font = 1, srt = labPosAngle, 
            cex = labPosSize, adj = 0.1)
    }
    if (showLegend) {
        par(mar = c(0, 0.5, 1, 0), xpd = TRUE)
        plot(NULL, ylab = "", xlab = "", xlim = 0:1, ylim = 0:1, 
            axes = FALSE)
        lep = legend("topleft", legend = names(col), col = col, 
            bty = "n", border = NA, xpd = TRUE, text.font = 1, 
            pch = 16, xjust = 0, yjust = 0, cex = legendTxtSize, 
            y.intersp = 1.5, x.intersp = 1, pt.cex = 1.2 * legendTxtSize, 
            ncol = ceiling(length(col)/4))
        x_axp = 0 + lep$rect$w
        if (!showDomainLabel) {
            if (length(domain_cols) <= 4) {
                n_col = 1
            }
            else {
                n_col = (length(domain_cols)%/%4) + 1
            }
            lep = legend(x = x_axp, y = 1, legend = names(domain_cols), 
                col = domain_cols, border = NA, ncol = n_col, 
                pch = 15, xpd = TRUE, xjust = 0, bty = "n", 
                cex = legendTxtSize, title = "Domains", title.adj = 0, 
                pt.cex = 1.2 * legendTxtSize)
        }
    }
    if (printCount) {
        print(prot.snp.sumamry[, .(pos, conv, count, Variant_Classification)][order(pos)])
    }
}

In [9]:
# %%R -i maf_path
# cosmic_maf = read.maf(maf = maf_path)

# lollipopPlot(
#   maf = cosmic_maf,
#   gene = 'MT-CO1',
#   AACol = "HGVSp_Short",
#   showMutationRate = TRUE,
#   # labelPos = c(687, 678),
# )

### Load in data

In [ ]:
%%R -i maf_path
cosmic_maf = read.maf(maf = maf_path)  # removeDuplicatedVariants=FALSE if needed, but I don't think it is

-Reading
-Validating
-Silent variants: 13 
-Summarizing
-Processing clinical data
--Missing clinical data
-Finished in 0.041s elapsed (0.022s cpu) 


In [39]:
# Load the MAF file as a DataFrame
df = pd.read_csv(maf_path, sep='\t', comment='#', low_memory=False)

# Select the desired columns
selected_columns = ["Hugo_Symbol", "vcf_id", "Tumor_Sample_Barcode", "Protein_position", "HGVSp_Short", "Variant_Classification", "DP", "NS"]
df = df[selected_columns]
df['Protein_position'] = df['Protein_position'].str.split('/').str[0].str.split('-').str[-1].astype(int)  # take the 2nd number in the dash
df['DP'] = df['DP'].astype(int)
df.rename(columns={"DP": "count", "Protein_position": "pos"}, inplace=True)

# Display the first few rows
df.head()

,Hugo_Symbol,vcf_id,Tumor_Sample_Barcode,pos,HGVSp_Short,Variant_Classification,count,NS
0,WASH4P,ENST00000326592:c.1235C>T,TUMOR,192,p.T192=,Silent,2,1
1,MT-CO1,ENST00000361624:c.924T>C,TUMOR,308,p.A308=,Silent,2,1
2,MT-CO1,ENST00000361624:c.1439G>A,TUMOR,480,p.R480Q,Missense_Mutation,2,1
3,MT-CO1,ENST00000361624:c.922G>A,TUMOR,308,p.A308T,Missense_Mutation,8,1
4,MT-CO1,ENST00000361624:c.1477G>A,TUMOR,493,p.E493K,Missense_Mutation,3,1


### Sort genes

In [40]:
# Sort genes in descending order
if sorting_method == "total_counts":
    # sort based on total counts
    sorted_genes = df.groupby("Hugo_Symbol")["count"].sum().sort_values(ascending=False)
    print("Gene: Counts")
elif sorting_method == "number_of_distinct_variants":
    # sort based on number of distinct variants
    sorted_genes = df["Hugo_Symbol"].value_counts()
    print("Gene: Number of Distinct Variants")
elif sorting_method == "total_samples":
    # sort based on number of distinct samples
    sorted_genes = df.groupby("Hugo_Symbol")["NS"].nunique().sort_values(ascending=False)
    print("Gene: Number of Distinct Samples")

sorted_genes_dict = sorted_genes.to_dict()
sorted_genes_dict

Gene: Number of Distinct Variants


{'MT-CO1': 21,
 'MT-ATP6': 16,
 'MT-ND2': 10,
 'NOC2L': 2,
 'WASH4P': 1,
 'MT-ND1': 1,
 'MT-ND6': 1}

### Loop through genes and create plots

In [41]:
os.makedirs(out_dir, exist_ok=True)

number_of_genes = sys.maxsize if number_of_genes is True else number_of_genes
show_mutations = sys.maxsize if show_mutations is True else show_mutations

# Loop through each gene in sorted order
for i, gene in enumerate(sorted_genes.index[:number_of_genes]):
    # Create a temporary DataFrame for the current gene
    temp_df = df[df['Hugo_Symbol'] == gene].copy()
    temp_df = temp_df[temp_df['Variant_Classification'] != "Silent"]  # these aren't included in the plot anyways, and this way they won't get in the way of position selection
    temp_df = temp_df.sort_values(by="count", ascending=False)
    
    positions = temp_df['pos'].tolist()[:show_mutations] if show_mutations else None

    plot_path = os.path.join(out_dir, f"lollipop_plot_{i+1}_{gene}.pdf")
    
    # Pass the data and gene to the R global environment
    globalenv['gene'] = gene
    globalenv['plot_path'] = plot_path
    globalenv['positions'] = positions

    # my_data = temp_df[['pos', 'count']]  # only if using non-MAF
    # globalenv['my_data'] = my_data

    # Execute the multi-line R code: open a PDF, plot, and close the PDF device
    r_code = '''
    pdf(plot_path)
    lollipopPlot_custom(
        maf = cosmic_maf,
        gene = gene,
        AACol = "HGVSp_Short",
        showMutationRate = FALSE,
        labelPos = positions,
        displayed_counts = displayed_counts
    )
    dev.off()
    '''
    # r_code = '''
    # pdf(plot_path)
    # lollipopPlot(data = my_data, gene = gene)
    # dev.off()
    # '''
    try:
        r(r_code)
        print(f"Plot saved for {gene} at {plot_path}")
    except Exception as e:
        print(f"Error generating plot for {gene}: {e}")
        continue


Plot saved for MT-CO1 at /Users/joeyrich/Desktop/local/RLSRWP_2025/data/vk_count_out_fig1/analysis/lollipop_plots4/lollipop_plot_1_MT-CO1.pdf
Plot saved for MT-ATP6 at /Users/joeyrich/Desktop/local/RLSRWP_2025/data/vk_count_out_fig1/analysis/lollipop_plots4/lollipop_plot_2_MT-ATP6.pdf
Plot saved for MT-ND2 at /Users/joeyrich/Desktop/local/RLSRWP_2025/data/vk_count_out_fig1/analysis/lollipop_plots4/lollipop_plot_3_MT-ND2.pdf
Plot saved for NOC2L at /Users/joeyrich/Desktop/local/RLSRWP_2025/data/vk_count_out_fig1/analysis/lollipop_plots4/lollipop_plot_4_NOC2L.pdf


R[write to console]: Error in maftools:::.getdomains(geneID = gene, refSeqID = refSeqID, proteinID = proteinID) : 
  Structure for protein WASH4P not found.



Error generating plot for WASH4P: Error in maftools:::.getdomains(geneID = gene, refSeqID = refSeqID, proteinID = proteinID) : 
  Structure for protein WASH4P not found.



In [15]:
# %%R
# my_data = data.frame(pos = sample.int(912, 15, replace = TRUE), count = sample.int(30, 15, replace = TRUE))
# head(my_data)
# # pdf("lollipop_plot.pdf")
# lollipopPlot(data = my_data, gene = "DNMT3A")
# # dev.off()